In [19]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

class DecisionTree:
    def __init__(self):
        self.tree = None

    def fit(self, X, y):
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        self.tree = self._build_tree(X, y_encoded)

    def _gini_index(self, groups, classes):
        total_instances = float(sum(len(group) for group in groups))
        gini = 0.0
        for group in groups:
            size = float(len(group))
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            gini += (1.0 - score) * (size / total_instances)
        return gini

    def _split(self, index, value, dataset):
        left, right = [], []
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    def _get_split(self, dataset):
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        for index in range(len(dataset[0])-1):
            for row in dataset:
                groups = self._split(index, row[index], dataset)
                gini = self._gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        return {'index':b_index, 'value':b_value, 'groups':b_groups}

    def _to_terminal(self, group):
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count)

    def _split_node(self, node, max_depth, min_size, depth):
        left, right = node['groups']
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self._to_terminal(left + right)
            return
        if depth >= max_depth:
            node['left'], node['right'] = self._to_terminal(left), self._to_terminal(right)
            return
        if len(left) <= min_size:
            node['left'] = self._to_terminal(left)
        else:
            node['left'] = self._get_split(left)
            self._split_node(node['left'], max_depth, min_size, depth+1)
        if len(right) <= min_size:
            node['right'] = self._to_terminal(right)
        else:
            node['right'] = self._get_split(right)
            self._split_node(node['right'], max_depth, min_size, depth+1)

    def _build_tree(self, X, y, max_depth=3, min_size=2):
        dataset = np.column_stack([X, y])
        root = self._get_split(dataset)
        self._split_node(root, max_depth, min_size, 1)
        return root

    def _print_tree(self, node, depth=0):
        if isinstance(node, dict):
            print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
            self._print_tree(node['left'], depth+1)
            self._print_tree(node['right'], depth+1)
        else:
            print('%s[%s]' % ((depth*' ', node)))

    def print_tree(self):
        self._print_tree(self.tree)

# 训练数据集
X_train = np.array([
    [1, 1, 0, 0, 1],
    [0, 0, 0, 1, 1],
    [1, 1, 0, 0, 1],
    [1, 1, 0, 0, 1],
    [1, 0, 0, 1, 0],
    [1, 0, 1, 0, 0]
])
y_train = np.array(['yes', 'no', 'yes', 'yes', 'no', 'no'])

# 构建并训练决策树
tree = DecisionTree()
tree.fit(X_train, y_train)

# 打印决策树
tree.print_tree()


[X2 < 1.000]
 [X1 < 0.000]
  [0]
  [0]
 [X1 < 1.000]
  [1]
  [1]


In [16]:
import numpy as np

class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature          # 用于分裂的特征
        self.threshold = threshold      # 分裂阈值
        self.left = left                # 左子树
        self.right = right              # 右子树
        self.value = value              # 叶节点的类别预测值

def gini_index(groups, classes):
    """
    计算基尼不纯度
    """
    total_instances = float(sum(len(group) for group in groups))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / total_instances)
    return gini

def test_split(index, value, dataset):
    """
    根据特征和阈值分割数据集
    """
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    """
    获取最佳的分裂点
    """
    class_values = list(set(row[-1] for row in dataset))
    best_index, best_value, best_score, best_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_index, best_value, best_score, best_groups = index, row[index], gini, groups
    return {'index': best_index, 'value': best_value, 'groups': best_groups}

def to_terminal(group):
    """
    将组中最常见的输出类别作为节点的值返回
    """
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    """
    递归构建决策树
    """
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
    """
    构建决策树
    """
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def print_tree(node, depth=0):
    """
    打印决策树
    """
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

# 训练数据集
dataset = [
    [1, 1, 0, 0, 1, 'yes'],
    [2, 0, 0, 1, 1, 'no'],
    [3, 1, 0, 0, 1, 'yes'],
    [4, 1, 0, 0, 1, 'yes'],
    [5, 1, 0, 1, 0, 'no'],
    [6, 1, 0, 1, 0, 'no']
]

# 构建决策树
tree = build_tree(dataset, 2, 1)

# 打印决策树
print_tree(tree)


[X4 < 1.000]
 [X1 < 1.000]
  [yes]
  [yes]
 [X1 < 2.000]
  [no]
  [no]


In [20]:
import numpy as np
from sklearn.cluster import KMeans

# 表A5-1中的数据，只取属性列，并去掉重复的序号列  
data = np.array([
    [2, 10],
    [2, 5],
    [8, 4],
    [5, 8],
    [7, 5],
    [6, 4],
    [1, 2],
    [4, 9]
])

# 初始聚类中心（序号1、4、7对应的点）  
initial_centers = data[[0, 3, 6]]

# 为了模拟第一次迭代，我们需要手动计算每个点到初始聚类中心的距离  
def euclidean_distance(point, center):
    return np.sqrt(np.sum((point - center) ** 2))

# 手动分配点到最近的簇（第一次迭代）  
def assign_clusters(data, centers):
    labels = np.zeros(len(data), dtype=int)
    for i, point in enumerate(data):
        distances = [euclidean_distance(point, center) for center in centers]
        labels[i] = np.argmin(distances)
    return labels

# 第一次迭代：分配点到簇  
labels_first_iter = assign_clusters(data, initial_centers)

# 手动更新聚类中心（第一次迭代）  
def update_centers(data, labels, num_clusters):
    new_centers = np.zeros((num_clusters, 2))
    for i in range(num_clusters):
        cluster_points = data[labels == i]
        if len(cluster_points) > 0:
            new_centers[i] = np.mean(cluster_points, axis=0)
    return new_centers

# 第一次迭代：更新聚类中心  
new_centers_first_iter = update_centers(data, labels_first_iter, 3)

print("第一次迭代后的聚类中心:")
print(new_centers_first_iter)

# 使用scikit-learn的KMeans获取最终聚类结果  
kmeans = KMeans(n_clusters=3, init=initial_centers, n_init=1, random_state=42)
kmeans.fit(data)

print("最终的聚类中心:")
print(kmeans.cluster_centers_)

# 预测每个样本所属的簇（最终）  
final_labels = kmeans.predict(data)
print("最终的簇标签:")
print(final_labels)


第一次迭代后的聚类中心:
[[ 2.  10. ]
 [ 6.   6. ]
 [ 1.5  3.5]]
最终的聚类中心:
[[3.66666667 9.        ]
 [7.         4.33333333]
 [1.5        3.5       ]]
最终的簇标签:
[0 2 1 0 1 1 2 0]
